In [217]:
import json
from json_logic import jsonLogic
from json_logic.builtins import BUILTINS, op_var, op_log
from functools import reduce

In [218]:
def courses(data=None, *_ignored):
    return list(filter(None, op_var(data, "courses")))


def course(data=None, code="", number="", *_ignored):
    for course in courses(data):
        if course["code"] == code and course["number"] == number:
            return course
    return None


def units(data=None, *courses):
    flatten_list = lambda *lists: reduce(lambda a, b: a + b, lists, [])
    courses = flatten_list(*courses)
    
    total = 0
    for course in filter(None, courses):
        total += course["units"]
    return total

def consent(data=None, *consenter):
    op_log("consent", consenter)
    return True

ops = {
    **BUILTINS,
    "courses": courses,
    "course": course,
    "units": units,
    "consent": consent,
}

In [219]:
data = {
    "courses": [
        {"code": "CPSC", "number": "329", "gpv": 2.7, "semester": "W2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "526", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "313", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "271", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "STAT", "number": "205", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

In [220]:
rule = {
    "some": [
        {"course": ["CPSC", "351"]},
        {
            "and": [
                {
                    ">=": [
                        {
                            "units": [
                                {"course": ["CPSC", "219"]},
                                {"course": ["CPSC", "233"]},
                                {"course": ["CPSC", "235"]},
                            ]
                        },
                        3,
                    ]
                },
                {
                    ">=": [
                        {
                            "units": [
                                {"course": ["MATH", "271"]},
                                {"course": ["MATH", "273"]},
                                {"course": ["MATH", "315"]},
                            ]
                        },
                        3,
                    ]
                },
                {
                    ">=": [
                        {
                            "units": [
                                {"course": ["STAT", "205"]},
                                {"course": ["STAT", "213"]},
                                {"course": ["STAT", "321"]},
                            ]
                        },
                        3,
                    ]
                },
            ]
        },
    ]
}

In [221]:
a = jsonLogic(rule, data, ops)
bool(a)

False

In [222]:
rule = {"units": {"courses": {"log": "apple"}}}

jsonLogic(rule, data, operations=ops)

"apple"


24.0

```json
{
  "cid": "160417",
  "code": "CPSC",
  "number": "219",
  "faculty": "SC",
  "departments": [
    "CPSC"
  ],
  "units": 3.0,
  "prereq": "CPSC 217 or DATA 211.",
  "antireq": "Credit for CPSC 219 and any of 233, 235, ENEL 497 or ENCM 493 will not be allowed.",
  "coreq": null
}
```

In [223]:
rule = {
    "cid": "160417",
    "prereq": {
        "or": [
            {"course": ["CPSC", "217"]}, 
            {"course": ["DATA", "211"]}
        ]
    },
    "antireq": {
        "and": [
            {"course": ["CPSC", "219"]},
            {
                "or": [
                    [
                        {"course": ["CPSC", "233"]},
                        {"course": ["CPSC", "235"]},
                        {"course": ["ENEL", "497"]},
                        {"course": ["ENCM", "493"]},
                    ],
                    True
                ]
            },
        ]
    },
    "coreq": None,
}

json.dumps(rule)

'{"cid": "160417", "prereq": {"or": [{"course": ["CPSC", "217"]}, {"course": ["DATA", "211"]}]}, "antireq": {"and": [{"course": ["CPSC", "219"]}, {"or": [[{"course": ["CPSC", "233"]}, {"course": ["CPSC", "235"]}, {"course": ["ENEL", "497"]}, {"course": ["ENCM", "493"]}], true]}]}, "coreq": null}'

In [224]:
data = {
    "courses": [
        {"code": "CPSC", "number": "217", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "233", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rule["prereq"], data, operations=ops)
b = jsonLogic(rule["antireq"], data, operations=ops)

bool(a), bool(b)

(True, True)

```json
{
  "cid": "164005",
  "code": "CPSC",
  "number": "405",
  "faculty": "SC",
  "departments": [
    "CPSC"
  ],
  "units": 3.0,
  "prereq": "SENG 300 or 301.",
  "antireq": "Credit for CPSC 405 and ENTI 415 will not be allowed.",
  "coreq": null
}
```

In [225]:
rule = {
    "cid": "164005",
    "prereq": {
        "or": [
            {"course": ["SENG", "300"]}, 
            {"course": ["SENG", "301"]}
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "405"]},
            {"course": ["ENTI", "415"]},
        ]
    },
    "coreq": None,
}

json.dumps(rule)

'{"cid": "164005", "prereq": {"or": [{"course": ["SENG", "300"]}, {"course": ["SENG", "301"]}]}, "antireq": {"or": [{"course": ["CPSC", "405"]}, {"course": ["ENTI", "415"]}]}, "coreq": null}'

In [226]:
data = {
    "courses": [
        {"code": "SENG", "number": "300", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        # {"code": "CPSC", "number": "405", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rule["prereq"], data, operations=ops)
b = jsonLogic(rule["antireq"], data, operations=ops)


bool(a), bool(b)

(True, False)

In [227]:
rule = {"all": [[False, False], {"==": [{"var": ""}, True]}]}
rule = {"and": [None, 6, 9]}

data = None

a = jsonLogic(rule, data, operations=ops)
bool(a)

False